In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Train MetaData**

**Exploring**

In [ ]:
train_csv_path = '/kaggle/input/tensorflow-great-barrier-reef/train.csv'
train_df = pd.read_csv(train_csv_path)

train_df

In [ ]:
#Video frames in sequence wrt each video
#Creating df of each video

video0 = train_df[train_df['video_id']==0]
video1 = train_df[train_df['video_id'] ==1]
video2 = train_df[train_df['video_id'] == 2]

In [ ]:
#Each video has frames in sequence
video0 = video0.sort_values(by=['video_frame'])
video1 = video1.sort_values(by=['video_frame'])
video2 = video2.sort_values(by=['video_frame'])

In [ ]:
video0

In [ ]:
video1

In [ ]:
video2

**Annotations**

In [ ]:
#Percentage of no annotations in video
def no_annot_perc(df):
    no_annot = sum(df['annotations'] == '[]')
    total = len(df)
    
    percentage = (no_annot/total)*100
    print(f"Percentage of no annotations in video {df.iloc[0,0]}: {round(percentage,2)} ")

In [ ]:
no_annot_perc(video0)
no_annot_perc(video1)
no_annot_perc(video2)

Majority of frames of each video contains no annotations

**Individual Frames with annotations**

In [ ]:
import cv2
import ast
import os
import matplotlib.pyplot as plt
from PIL import Image,ImageDraw
%matplotlib inline

In [ ]:
#Plot single image from Df
def plot_image(df,im_id):
    #video_id
    vid_id = df.iloc[im_id,0]
    frame_number = df.iloc[im_id,2]
    
    vid_path = f"/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{vid_id}"
    img_file = f"{frame_number}.jpg"
    
    img_path = os.path.join(vid_path,img_file)
    
    annotations = df.iloc[im_id,5]
    
    if annotations == "[]":
        #img = cv2.imread(img_path)
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = Image.open(img_path)
        
    else:
        annotations = ast.literal_eval(annotations)
        img = Image.open(img_path)
        #img = cv2.imread(img_path)
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        length = len(annotations)
        for i in range(len(annotations)):
            obj = annotations[i]
            x1 = obj['x']
            y1 = obj['y']
            x2 = x1 + obj['width']
            y2 = y1 + obj['height']
            
            #cv2.rectangle(img,(x1,x2),(x2,y2),color=(255,0,0),thickness=2)
            
            img1 = ImageDraw.Draw(img)
            img1.rectangle([x1,y1,x2,y2],outline="red",width=2)
            
            print(f"Height: {obj['height']}, Width: {obj['width']}")
            print(f"Number of Detections: {length}")
            print(f"X1,y1: {(x1,y1)} X2,y2: {(x2,y2)}")
            
        
            
    
    #pil_img = Image.fromarray(img)
    
    #plt.figure(figsize=(16,10))
    #plt.imshow(img)
    return img
    
            
        

In [ ]:
img = plot_image(video0,70)
img

In [ ]:
img = plot_image(video0,106)
img

**Creating VideoStream**

In [ ]:
!mkdir "/kaggle/working/videos"

In [ ]:
from glob import glob

In [ ]:
image_path = f"/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/0.jpg"
img = cv2.imread(image_path)
img.shape

In [ ]:
def videostream(df):
    video_id = df.iloc[0,0]
    
    images_path = f"/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{video_id}/*.jpg"
    video_dest = f"/kaggle/working/videos/video_{video_id}.avi"
    
    #Video frames
    images = glob(images_path)
    images.sort()
    frame_height,  frame_width, channels = cv2.imread(images[0]).shape
    fps = 24
    
    #Creating Video Writer
    fourcc = cv2.VideoWriter_fourcc(*'MPEG')
    vid_writer = cv2.VideoWriter(video_dest,fourcc,fps,(frame_width,frame_height))
    
    ##Draw annotations on frames
    for i in range(len(images)):
        img_path = images[i]
        annotations = df.iloc[i,5]
        
        if annotations == "[]":
            img  = Image.open(img_path)
            img = np.array(img)
            
        else:
            annotations = ast.literal_eval(annotations)
            img = Image.open(img_path)
            
            
            length = len(annotations)
            for j in range(length):
                obj = annotations[j]
                x1 = obj['x']
                y1 = obj['y']
                x2 = x1 + obj['width']
                y2 = y1 + obj['height']
                
                img1 = ImageDraw.Draw(img)
                img1.rectangle([x1,y1,x2,y2],outline="red",width=2)
                
            img = np.array(img)
            
        vid_writer.write(img)
        
    vid_writer.release()
                
                

In [ ]:
videostream(video0)

In [ ]:
# Install ffmpeg for video compression
%cd /kaggle/working

! tar xvf ../input/ffmpegstaticbuild/ffmpeg-git-amd64-static.tar.xz

import subprocess

FFMPEG_BIN = "./ffmpeg-git-20220108-amd64-static/ffmpeg"

In [ ]:
#
AVI2MP4 = "-ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4"

command = f"{FFMPEG_BIN} -i /kaggle/working/videos/video_0.avi {AVI2MP4} /kaggle/working/videos/videos_0.mp4"
subprocess.call(command, shell=True)

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

play("/kaggle/working/videos/videos_0.mp4")

Notebook Incomplete.

CV2 seems to write video in BGR(Not certain)